In [2]:
import torch
import torch.nn as nn
from kasim_model import Kasim_Model
from kasim_tokenizer import Tokenizer   

k_tokenizer=Tokenizer("tokenizer.json")
sentence = "the capital of united states"
tokens = k_tokenizer.encode(sentence)

# tokens tek boyutlu: torch.Size([20])
#tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)  # shape: (1, 20)
tokens

tensor([ 0, 61,  1, 61,  2, 61,  3, 61,  4, 58])

In [3]:
import torch
from kasim_model import Kasim_Model
torch.manual_seed(1)
k_model = Kasim_Model(vocab_size=len(k_tokenizer.vocab), embed_dim=12, context_length=32, num_heads=4, num_layers=8)
out= k_model(tokens)
out.shape


torch.Size([1, 10, 64])

In [4]:
with open ("text.txt","r") as f:
    text=f.read()

text    

'the capital of the united states is not london. the capital of france is paris, and berlin is the capital of germany. rome is in italy, \n\nmadrid is in spain, and lisbon is in portugal. the capital of the united kingdom is not paris, and the capital of the united states is not berlin. \nalthough these places are often mentioned together, although these capitals are often mentioned together, although these are often mentioned together, \neach country has its own capital, and each country has its own city, and each capital has its own identity, and each capital has its own history. washington \nis the capital of the united states, and london is the capital of the united kingdom. paris is known for art and fashion, and berlin is known for art and \nhistory, and rome is known for art and history, and madrid is known for culture and history, and lisbon is known for culture and art. rome is rich with culture, \nrome is rich with history, rome is rich with art, and madrid is rich with art a

In [5]:
token_id=k_tokenizer.encode(text)
len(token_id)
len(text)

4104

In [6]:
token_id = token_id.detach().cpu().numpy().tolist()

In [7]:
from text_dataset import TextDataset

stride = 12
context_length=32
dataset=TextDataset(token_id,context_length,stride)
len(dataset.inputs)

131

In [8]:
# model parameters count
parameters_count = sum(p.numel() for p in k_model.parameters())
print(parameters_count)

# model architecture
print(k_model)

23104
Kasim_Model(
  (embedding): Embedding(64, 12)
  (pos_embed): Embedding(32, 12)
  (layers): Sequential(
    (0): KasimDecoderBlock(
      (self_attention): Kasim_Multi_Head_Attention(
        (multi_head_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=12, out_features=12, bias=True)
        )
        (projection): Linear(in_features=12, out_features=12, bias=True)
      )
      (norm1): KasimRMSNorm()
      (mlp): KasimMLP(
        (gate_proj): Linear(in_features=12, out_features=48, bias=True)
        (up_proj): Linear(in_features=12, out_features=48, bias=True)
        (down_proj): Linear(in_features=48, out_features=12, bias=True)
        (gelu): KasimGelu()
      )
      (norm2): KasimRMSNorm()
    )
    (1): KasimDecoderBlock(
      (self_attention): Kasim_Multi_Head_Attention(
        (multi_head_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=12, out_features=12, bias=True)
 

In [9]:
k_model.embedding.weight.shape

torch.Size([64, 12])

In [10]:
from torch.utils.data import DataLoader
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

batch = next(iter(dataloader))
inputs = batch["input_ids"]   # zaten tensor
targets = batch["labels"]     # zaten tensor

out = k_model(inputs)
out = out.reshape(-1, out.size(-1))
targets = targets.reshape(-1)
loss = loss_fn(out, targets)
print(loss)


tensor(4.5460, grad_fn=<NllLossBackward0>)


In [11]:
optimizer=torch.optim.AdamW(k_model.parameters(),lr=1e-3)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
k_model.to(device)
epochs=500
for epoch in range(epochs):
    total_loss = 0.
    for batch in dataloader:
        inputs = batch["input_ids"].to(device)
        targets = batch["labels"].to(device)

        pred = k_model(inputs)
        pred = pred.reshape(-1, pred.size(-1))
        targets = targets.reshape(-1)

        loss = loss_fn(pred, targets)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, average loss: {avg_loss:.4f}")


Epoch 1, average loss: 4.3499
Epoch 2, average loss: 4.0036
Epoch 3, average loss: 3.6388
Epoch 4, average loss: 3.3463
Epoch 5, average loss: 3.0635
Epoch 6, average loss: 2.9429
Epoch 7, average loss: 2.8281
Epoch 8, average loss: 2.7108
Epoch 9, average loss: 2.5849
Epoch 10, average loss: 2.5430
Epoch 11, average loss: 2.4810
Epoch 12, average loss: 2.3911
Epoch 13, average loss: 2.3433
Epoch 14, average loss: 2.3017
Epoch 15, average loss: 2.2534
Epoch 16, average loss: 2.2349
Epoch 17, average loss: 2.2302
Epoch 18, average loss: 2.1499
Epoch 19, average loss: 2.1372
Epoch 20, average loss: 2.1519
Epoch 21, average loss: 2.0808
Epoch 22, average loss: 2.0890
Epoch 23, average loss: 2.0911
Epoch 24, average loss: 2.0413


KeyboardInterrupt: 

In [13]:
epochs = 200

for epoch in range(epochs):
    total_loss = 0.
    for batch in dataloader:
        inputs = batch["input_ids"]
        targets = batch["labels"]

        pred = k_model(inputs)
        pred = pred.reshape(-1, pred.size(-1))
        targets = targets.reshape(-1)

        loss = loss_fn(pred, targets)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, average loss: {avg_loss:.4f} ")


Epoch 1, average loss: 2.0500 
Epoch 2, average loss: 2.0166 
Epoch 3, average loss: 2.0180 
Epoch 4, average loss: 2.0176 
Epoch 5, average loss: 2.0167 
Epoch 6, average loss: 1.9749 
Epoch 7, average loss: 1.9866 
Epoch 8, average loss: 1.9771 
Epoch 9, average loss: 1.9837 
Epoch 10, average loss: 1.9179 
Epoch 11, average loss: 1.9382 
Epoch 12, average loss: 1.9197 
Epoch 13, average loss: 1.9055 
Epoch 14, average loss: 1.9465 
Epoch 15, average loss: 1.9418 
Epoch 16, average loss: 1.9218 
Epoch 17, average loss: 1.9026 
Epoch 18, average loss: 1.9179 
Epoch 19, average loss: 1.8623 
Epoch 20, average loss: 1.8660 
Epoch 21, average loss: 1.8883 
Epoch 22, average loss: 1.8957 
Epoch 23, average loss: 1.8586 
Epoch 24, average loss: 1.8643 
Epoch 25, average loss: 1.8627 
Epoch 26, average loss: 1.8727 
Epoch 27, average loss: 1.8347 
Epoch 28, average loss: 1.8243 
Epoch 29, average loss: 1.8690 
Epoch 30, average loss: 1.8199 
Epoch 31, average loss: 1.8313 
Epoch 32, average

KeyboardInterrupt: 

In [56]:
import torch

out=k_model(tokens)
probs=torch.softmax(out[-1],dim=-1)
max_prob,max_indext=torch.max(probs,dim=-1)
max_prob,max_indext,probs

(tensor([0.9996, 0.3144, 0.9993, 0.5676, 0.9997, 0.3654, 0.9996, 0.2352, 0.9903,
         0.8004], grad_fn=<MaxBackward0>),
 tensor([61,  1, 61,  2, 61,  2, 61,  3, 58, 61]),
 tensor([[2.3912e-09, 9.1006e-10, 4.1367e-11, 2.1925e-12, 5.4059e-09, 1.9880e-06,
          8.8059e-12, 2.9546e-10, 5.7932e-12, 2.9939e-12, 3.3423e-10, 1.0838e-08,
          7.6299e-15, 4.6250e-06, 4.7222e-07, 2.5737e-09, 8.5263e-11, 1.4422e-09,
          3.6366e-12, 5.4855e-12, 2.0818e-11, 9.3524e-10, 2.3824e-11, 7.4980e-11,
          6.4473e-13, 6.3438e-08, 7.0918e-08, 2.0496e-06, 8.5163e-10, 1.8009e-11,
          5.3389e-08, 1.8705e-12, 5.8771e-13, 1.9998e-10, 1.9882e-07, 2.6093e-11,
          1.7134e-15, 2.5912e-09, 1.2627e-10, 1.2760e-11, 2.3119e-12, 1.6592e-09,
          2.4493e-11, 2.6871e-10, 4.7252e-08, 1.6120e-09, 4.7048e-10, 1.5141e-13,
          1.1342e-11, 3.4257e-11, 6.2062e-10, 1.1431e-10, 1.8579e-13, 3.0879e-07,
          1.0523e-10, 2.3922e-10, 9.0388e-16, 3.1162e-06, 1.5734e-08, 4.0243e-04,
     

In [70]:
import torch

# Tokenize ve tensor al
new_tokens = k_tokenizer.encode("madrid is in")  # tensor

# Son token ekle
new_tokens = torch.cat([new_tokens, torch.tensor([61])], dim=0)  # (seq_len+1,)

# Batch dimension ekle
inputs = new_tokens.unsqueeze(0)  # (1, seq_len+1)

# Modeli çalıştır
out = k_model(inputs)  # (1, seq_len+1, vocab_size)

# Son token için olasılık ve tahmin
last_token_logits = out[:, -1, :]           # (1, vocab_size)
probs = torch.softmax(last_token_logits, dim=-1)
max_prob, max_index = torch.max(probs, dim=-1)

print("Max probability:", max_prob.item())
print("Predicted token id:", max_index.item())


Max probability: 0.8347604274749756
Predicted token id: 51


Generated text: madrid is in italy, madrid


C:\Users\kasim\AppData\Local\Temp\ipykernel_16396\2093987580.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(prompt_tokens).unsqueeze(0)   # batch dimension


In [71]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# Cihaz ayarı
device = "cuda" if torch.cuda.is_available() else "cpu"
k_model.to(device)

# Loss fonksiyonu
loss_fn = nn.CrossEntropyLoss()

# Optimizer ve scheduler
lr = 5e-4
optimizer = torch.optim.AdamW(k_model.parameters(), lr=lr, weight_decay=0.01)
epochs = 200
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# Dataloader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

for epoch in range(epochs):
    k_model.train()
    total_loss = 0.0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs = batch["input_ids"].to(device)
        targets = batch["labels"].to(device)

        optimizer.zero_grad()

        # Forward pass
        pred = k_model(inputs)
        pred = pred.view(-1, pred.size(-1))
        targets = targets.view(-1)

        # Loss
        loss = loss_fn(pred, targets)

        # Backward pass
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(k_model.parameters(), max_norm=1.0)

        # Optimizer step
        optimizer.step()

        total_loss += loss.item()

    # Scheduler step
    scheduler.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")


Epoch 1/200:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\kasim\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1/200: 100%|██████████| 5/5 [00:03<00:00,  1.40it/s]


Epoch 1, Average Loss: 0.2613


Epoch 2/200: 100%|██████████| 5/5 [00:02<00:00,  1.69it/s]


Epoch 2, Average Loss: 0.2677


Epoch 3/200: 100%|██████████| 5/5 [00:02<00:00,  1.71it/s]


...

Epoch 198, Average Loss: 0.2028


Epoch 199/200: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


Epoch 199, Average Loss: 0.2020


Epoch 200/200: 100%|██████████| 5/5 [00:04<00:00,  1.09it/s]

Epoch 200, Average Loss: 0.1915


In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
k_model.to(device)

# Loss ve optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(k_model.parameters(), lr=1e-3)

# Dataloader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Eğitim döngüsü
epochs = 10000  # Denemek için az tutuyoruz
for epoch in range(epochs):
    total_loss = 0
    k_model.train()
    
    for batch in dataloader:
        inputs = batch["input_ids"].to(device)
        targets = batch["labels"].to(device)
        
        optimizer.zero_grad()
        out = k_model(inputs)
        
        # Flatten batch & sequence
        out = out.view(-1, out.size(-1))
        targets = targets.view(-1)
        
        loss = loss_fn(out, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")


Epoch 1, Average Loss: 1.6622
Epoch 2, Average Loss: 1.6098
Epoch 3, Average Loss: 1.6292
Epoch 4, Average Loss: 1.6483
Epoch 5, Average Loss: 1.6216
Epoch 6, Average Loss: 1.6007
Epoch 7, Average Loss: 1.6193
Epoch 8, Average Loss: 1.6087
Epoch 9, Average Loss: 1.6219
Epoch 10, Average Loss: 1.5867
Epoch 11, Average Loss: 1.6110
Epoch 12, Average Loss: 1.6073
Epoch 13, Average Loss: 1.5868
Epoch 14, Average Loss: 1.5635
Epoch 15, Average Loss: 1.5650
Epoch 16, Average Loss: 1.5703
Epoch 17, Average Loss: 1.5735
Epoch 18, Average Loss: 1.5791
Epoch 19, Average Loss: 1.5753
Epoch 20, Average Loss: 1.5446
Epoch 21, Average Loss: 1.5729
Epoch 22, Average Loss: 1.5755
Epoch 23, Average Loss: 1.5573
Epoch 24, Average Loss: 1.5258
Epoch 25, Average Loss: 1.5334
Epoch 26, Average Loss: 1.5563
Epoch 27, Average Loss: 1.5907
Epoch 28, Average Loss: 1.5284
Epoch 29, Average Loss: 1.5244
Epoch 30, Average Loss: 1.5531
Epoch 31, Average Loss: 1.5223
Epoch 32, Average Loss: 1.5143
Epoch 33, Average

In [31]:
import torch

# Başlangıç prompt
prompt_tokens = k_tokenizer.encode("and the capital of the united states is")  # tensor değilse tensor yap
inputs = torch.tensor(prompt_tokens).unsqueeze(0)   # batch dimension

generated_tokens = prompt_tokens.tolist()  # çıktıları saklamak için liste

max_new_tokens = 5  # kaç token üretmek istediğin

for _ in range(max_new_tokens):
    out = k_model(inputs)                     # (1, seq_len, vocab_size)
    last_token_logits = out[:, -1, :]         # son token için logits
    probs = torch.softmax(last_token_logits, dim=-1)
    next_token = torch.argmax(probs, dim=-1)  # tahmin edilen token id

    # Token’ı listeye ekle
    generated_tokens.append(next_token.item())

    # Input’u güncelle
    inputs = torch.tensor(generated_tokens).unsqueeze(0)
    

# Sonuçları decode et
generated_text = k_tokenizer.decode(generated_tokens)
print("Generated text:", generated_text)


Generated text: and the capital of the united states is not berlin.


C:\Users\kasim\AppData\Local\Temp\ipykernel_13880\689734791.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(prompt_tokens).unsqueeze(0)   # batch dimension
